<a href="https://colab.research.google.com/github/Janderson1704/WEB-MAPA-SIGEF-BAHIA/blob/main/WEB_MAPA_SIGEF_BAHIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

!pip install folium geopandas requests


import folium
import geopandas as gpd
import requests
from folium.plugins import MousePosition
import IPython.display


centro_bahia = [-12.5797, -41.7007]


mapa_bahia = folium.Map(location=centro_bahia, zoom_start=7, control_scale=True)


folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri, Maxar, Earthstar Geographics',
    name='Esri.WorldImagery'
).add_to(mapa_bahia)

folium.TileLayer(
    tiles='https://{s}.tile.openstreetmap.fr/hot/{z}/{x}/{y}.png',
    attr='&copy; OpenStreetMap contributors',
    name='OpenStreetMap.HOT'
).add_to(mapa_bahia)


wms_url = "http://acervofundiario.incra.gov.br/i3geo/ogc.php?"
folium.raster_layers.WmsTileLayer(
    url=wms_url,
    name="Imóveis Certificados SIGEF - Bahia",
    layers="certificada_sigef_particular_ba",
    fmt="image/png",
    transparent=True,
    version="1.1.1",
    opacity=0.5
).add_to(mapa_bahia)


url_geojson = "https://acervofundiario.incra.gov.br/dados/geojson/sigef_bahia.geojson"

try:
    response = requests.get(url_geojson, verify=False)


    if response.status_code == 200:
        sigef_geojson = response.json()


        sigef_layer = folium.FeatureGroup(name="SIGEF - Bahia (Interativo)")


        for feature in sigef_geojson["features"]:
            properties = feature.get("properties", {})
            codigo_imovel = properties.get("codigo_imovel", "N/A")
            situacao = properties.get("situacao", "N/A")
            area_total = properties.get("area_total", "N/A")
            municipio = properties.get("municipio", "N/A")


            popup_content = f"""
                <b>Código do Imóvel:</b> {codigo_imovel}<br>
                <b>Situação:</b> {situacao}<br>
                <b>Área Total:</b> {area_total} ha<br>
                <b>Município:</b> {municipio}
            """
            popup = folium.Popup(popup_content, max_width=300)


            folium.GeoJson(
                feature,
                style_function=lambda x: {
                    "fillColor": "#ff7800",
                    "color": "black",
                    "weight": 2,
                    "fillOpacity": 0.5
                }
            ).add_child(popup).add_to(sigef_layer)


        sigef_layer.add_to(mapa_bahia)

    else:
        print(f"Erro ao acessar SIGEF: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Erro na requisição: {e}")


folium.LayerControl().add_to(mapa_bahia)
mapa_bahia.add_child(MousePosition())


mapa_bahia.save("mapa-sigef-bahia.html")


IPython.display.display(IPython.display.HTML('<a href="/content/mapa-sigef-bahia.html" target="_blank">Clique aqui para abrir o mapa</a>'))

Erro ao acessar SIGEF: 404


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'acervofundiario.incra.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


# Nova seção

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
